# preparation

In [4]:
import dashscope
import os

In [2]:
# get API key
# dashscope.api_key = os.getenv("DASHSCOPE_API_KEY")

api_key = os.environ.get('DASHSCOPE_API_KEY')
dashscope.api_key = 'sk-16f2c80d9d3e4d7d9cc2f6b43f22dfbd' # https://bailian.console.aliyun.com/?tab=model#/api-key
# dashscope.api_key ='sk-qzkddkwofxndanqzonvabhwqbpwganpxkuuoghtxgvmzygnt' # https://cloud.siliconflow.cn/me/account/ak

In [ ]:
# %pip install -q addict
# %pip install -q "modelscope==1.25.0"
# %pip install -U "modelscope[aigc]"
# from modelscope import AutoModel, AutoTokenizer
from modelscope import AutoModelForCausalLM, AutoTokenizer

# Cases

## 2-2 Mobile phone service subscription

In [ ]:
# use deepseek-v3 model
def get_completion(prompt, model="deepseek-v3"):
    messages = [
        {"role": "user", "content": prompt}
    ]
    response = dashscope.Generation.call(
        model=model,
        messages=messages,
        result_format="message",
        temperature=0, # 模型输出的随机性，0 表示随机性最小
    )
    return response.output.choices[0].message.content

### 使用提示词完成任务

In [ ]:
# system prompt
instructions = """
你的任务是识别用户对手机流量套餐产品的选择条件。
每种流量套餐产品包含三个属性：名称，月费价格，月流量。
根据用户输入，识别用户在上述三种属性上的需求是什么。
"""

# user prompt
input_text = """办个100G的套餐。"""

prompt = f"""
{instructions}
{input_text}
"""

print("==== Prompt ====")
print(prompt)
print("================")

# use model to get response
response = get_completion(prompt)
print(response)


### JSON format

In [ ]:
output_format = """以 JSON 格式输出"""

prompt=f"""
{instructions}
{output_format}
{input_text}
"""
response = get_completion(prompt)
print(response)

### CoT case

In [ ]:
instructions = """
给定一段用户与手机流量套餐客服的对话，。
你的任务是判断客服的回答是否符合下面的规范：

- 必须有礼貌
- 必须用官方口吻，不能使用网络用语
- 介绍套餐时，必须准确提及产品名称、月费价格和月流量总量。上述信息缺失一项或多项，或信息与事实不符，都算信息不准确
- 不可以是话题终结者

已知产品包括：

经济套餐：月费50元，月流量10G
畅游套餐：月费180元，月流量100G
无限套餐：月费300元，月流量1000G
校园套餐：月费150元，月流量200G，限在校学生办理

"""

output_format = """
如果符合规范，输出：Y
如果不符合规范，输出：N
"""

context = """
用户：你们有什么流量大的套餐
客服：亲，我们现在正在推广无限套餐，每月300元就可以享受1000G流量，您感兴趣吗？
"""

# cot=""
cot="请一步一步分析对话"

prompt=f"""
{instructions}
{cot}
{output_format}
{context}
"""

response=get_completion(prompt)
print(response)

### 使用Prompt调优Prompt

In [ ]:
user_prompt = """
做一个手机流量套餐的客服代表，叫小瓜。可以帮助用户选择最合适的流量套餐产品。可以选择的套餐包括：
经济套餐，月费50元，10G流量；
畅游套餐，月费180元，100G流量；
无限套餐，月费300元，1000G流量；
校园套餐，月费150元，200G流量，仅限在校生。"""

instruction = """
你是一名专业的提示词创作者。你的目标是帮助我根据需求打造更好的提示词。

你将生成以下部分：
提示词：{根据我的需求提供更好的提示词}
优化建议：{用简练段落分析如何改进提示词，需给出严格批判性建议}
问题示例：{提出最多3个问题，以用于和用户更好的交流}
"""

prompt = f"""
{instruction}
{user_prompt}
"""

response = get_completion(prompt)
print(response)


## 2-3 DeepSeek-r1-7b

https://lingma.aliyun.com
Qwen3-Coder-480B-A35B-Instruct
480B 完整尺寸的模型参数大小
A35B，激活的参数量是35B

https://modelscope.cn/search?search=deepseek-r1
7B => GPU
1.5B => CPU 内存

https://modelscope.cn/models/deepseek-ai/DeepSeek-R1-Distill-Qwen-7B

**模型下载**

from modelscope import snapshot_download
snapshot_download('deepseek-ai/DeepSeek-R1-Distill-Qwen-7B', cache_dir="/root/autodl-tmp/models")

deepseek-ai/DeepSeek-R1-Distill-Qwen-7B => Model ID

方法1：vllm进行部署
方法2：python 调用部署好的模型
方法3：ollama

{'response': '<think>\n\n</think>\n\n您好！我是由中国的深度求索（DeepSeek）公司开发的智能助手DeepSeek-R1。我是一个AI assistant，通过机器学习技术来模拟人类思考和判断的能力。我理解用户的问题会通过这些正式的、诚实且专业的语言形式呈现，我会以友好、理性的态度为您提供详细的回答。'}

封装成API接口，给其他人用
方法1：fastapi
方法2：flask

约束1：模型尺寸，不会太大
约束2：上网环境

感觉对于我们学员来说，私有化部署是个熟悉使用模型的过程，有助于以后实战微调

deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B 与 Ollama运行deepseek-r1:1.5b是一样的吗？
是一样的参数

自己在魔搭平台下的大模型怎么跑？只能用ollama吗
方法1：ollama
方法2：vllm 
方法3：python调用大模型

服务器上怎么部署 调用
Step1，通过modelscope下载对应的大模型

**模型下载**

from modelscope import snapshot_download
snapshot_download('deepseek-ai/DeepSeek-R1-Distill-Qwen-7B', cache_dir="/root/autodl-tmp/models")


In [ ]:
# %pip install -q modelscope
# from modelscope import snapshot_download

# model_dir = snapshot_download(
# 	'deepseek-ai/DeepSeek-R1-Distill-Qwen-7B',
# 	cache_dir=r"d:\Job_hunt\2025Courses_LLM\models\deepseek"
# )
# print(f"Model downloaded to: {model_dir}")

In [ ]:
# %pip install -q addict
# %pip install -q "modelscope==1.25.0"
# %pip install -U "modelscope[aigc]"
# from modelscope import AutoModel, AutoTokenizer
from modelscope import AutoModelForCausalLM, AutoTokenizer

ImportError: cannot import name 'AutoModelForCausalLM' from 'modelscope' (e:\Python\python3.12\Lib\site-packages\modelscope\__init__.py)

: 

In [ ]:
model_name=r"d:\Job_hunt\2025Courses_LLM\models\deepseek\deepseek-ai\DeepSeek-R1-Distill-Qwen-7B"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"  # cuda
)

tokenizer = AutoTokenizer.from_pretrained(model_name)

prompt = "帮我写一个二分查找法"
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt}
]

text = tokenizer.apply_chat_template(
    messages,
    tokenizer=False,
    add_generation_prompt=True
)

model_inputs = tokenizer(
    [text],
    return_tensors="pt"
).to(model.device)

generation_ids = model.generate(
    **model_inputs,
    max_new_tokens=1024
)
generation_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs["input_ids"], generation_ids)
]

response = tokenizer.batch_decode(
    generation_ids,
    skip_special_tokens=True,
    clean_up_tokenization_spaces=False
)[0]
print(response)

## ollama

In [ ]:
# 本地模型推理（无需 HTTP 和 Ollama 服务）
import torch


In [ ]:
def query_local_model(prompt, model_path=None):
    """直接加载本地下载的模型进行推理，无需 Ollama 服务"""
    if model_path is None:
        model_path = r"d:\Job_hunt\2025Courses_LLM\models\deepseek\deepseek-ai\DeepSeek-R1-Distill-Qwen-7B"
    
    # 加载模型和分词器
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        torch_dtype="auto",
        device_map="auto"  # 自动选择 GPU/CPU
    )
    
    # 构建对话消息
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
    
    # 应用聊天模板
    text = tokenizer.apply_chat_template(
        messages,
        tokenizer=False,
        add_generation_prompt=True
    )
    
    # 分词和生成
    inputs = tokenizer([text], return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=512)
    gen_ids = [out[len(inp):] for inp, out in zip(inputs["input_ids"], outputs)]
    response = tokenizer.batch_decode(gen_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    
    return response

# 使用本地模型推理
response = query_local_model("请用一句话介绍一下你自己。")
print("=== 本地模型推理结果 ===")
print(response)


ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=11434): Max retries exceeded with url: /api/generate (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000015BAF107E90>: Failed to establish a new connection: [WinError 10061] 由于目标计算机积极拒绝，无法连接。'))